In [40]:
import random
import numpy as np
import sympy as sp

In [41]:
# Here is the proposed function
def sympy_lambdify_expression(
        sympy_variables: list[sp.Symbol] | tuple[sp.Symbol, ...],
        sympy_expression: sp.Expr):
    return sp.lambdify(sympy_variables, sympy_expression, 'numpy')

In [42]:
sympy_symbols = sp.symbols('x_1 x_2 x_3')
sympy_symbols # this is a tuple

(x_1, x_2, x_3)

In [43]:
list(sympy_symbols)

[x_1, x_2, x_3]

So `list()` on a tuple turns the tuple into a list?

In [44]:
underlying_function_1 = sympy_symbols[0] + sympy_symbols[1]**2 - sp.sin(sympy_symbols[2])
underlying_function_1

x_1 + x_2**2 - sin(x_3)

In [45]:
sympy_function = sympy_lambdify_expression(sympy_symbols, underlying_function_1)
sympy_function

<function _lambdifygenerated(x_1, x_2, x_3)>

Can we numerically evaluate it?

In [46]:
sympy_function(4, 4, 4)

20.756802495307927

Now, we try to not us all the variables to see if the function breaks:

In [47]:
underlying_function_2 = sympy_symbols[0]**sp.sin(sympy_symbols[0])
underlying_function_2

x_1**sin(x_1)

In [48]:
sympy_function_2 = sympy_lambdify_expression(sympy_symbols, underlying_function_2)
sympy_function_2

<function _lambdifygenerated(x_1, x_2, x_3)>

Looks like it *still* requires three input variables.

In [49]:
sympy_function_2(4)

TypeError: _lambdifygenerated() missing 2 required positional arguments: 'x_2' and 'x_3'

In [36]:
sympy_function_2(4, 4, 4)

0.3502349613017394

This is acceptable. We just have to remember that this happens when we go forward.

In [51]:
def sympy_nth_degree_polynomial(
    sympy_variable_x: sp.Symbol, 
    degree_of_polynomial: int,
    *coefficients) -> float:
    """
    ## Description:
    Create an nth-degree polynomial function of x.

    ## Parameters:
    - sympy_variable_x (sp.Symbol): The symbolic variable.
    - n (int): The degree of the polynomial.
    - *coefficients (float): The polynomial coefficients in order [a_0, a_1, ..., a_n].

    ## Returns:
    - sp.Expr: The symbolic polynomial expression.
    """
    if len(coefficients) != degree_of_polynomial + 1:
        raise ValueError(f"Expected {degree_of_polynomial + 1} coefficients, but got {len(coefficients)}.")
    
    try:
        return sum(coefficients[i] * sympy_variable_x**i for i in range(degree_of_polynomial + 1))
    except:
        FLOAT_ZERO = 0.
        return FLOAT_ZERO


def sympy_generate_random_function(
        sympy_symbols: sp.Symbol,
        depth: int) -> sp.Function:
    """
    ## Description:
    We use SymPy to generate a random function of a single variable. We generate
    the function using the `depth` parameter that determines how many iterations of 
    function composition we perform.

    ## Arguments:
    1. `sympy_symbols` (sp.Symbol)
    2. `depth` (int)
    """
    
    # (X): List of available (unary) functions:
    functions = [
        sympy_nth_degree_polynomial,
        ]
    
    # (X): Initalize a set object to store unique elements:
    used_independent_variables = set()

    # (X): Cast the SymPy tuple of symbols into a list:
    available_expressions = list(sympy_symbols)
    
    # (X): Initialize the expression tree generation:
    for _ in range(depth):

        # (X): In generating the expression tree, we choose either U(•) or B(•, •) operators:
        operation_type = np.random.choice(['unary', 'binary'])

        # (X): In U(•) operations, we need a single input:
        if operation_type == 'unary':

            # (X): Select a random expression:
            chosen_expression = np.random.choice(available_expressions)

            # (X): Compare if the expression *is or is not* a "primitive" SymPy symbol:
            if chosen_expression in sympy_symbols:

                # (X): If the selection *is* a primitive symbol, add this symbol as "used":
                used_independent_variables.add(chosen_expression)

            # (X): Choose a random function from the list `functions`:
            function_index = np.random.randint(0, len(functions))

            # (x): Ascertain the corresponding *function* from the list!
            function = functions[function_index]

            # (X.1): If the function is a nth-degree polynomial, we need to be fancy in handling it:
            if function == sympy_nth_degree_polynomial:

                # (3.1.1): Randomly choose the degree of the polynomial: (1 ≤ n ≤ 4)
                polynomial_degree = np.random.randint(2, 5)

                # (3.1.2): ...
                coefficients = np.round(np.random.uniform(-5, 5, size = polynomial_degree + 1)) 

                # (3.1.3):
                new_expression = function(chosen_expression, polynomial_degree, *coefficients)
            
            # (3.2): Otherwise, functions only come with finite and determined number of parameters:
            else:

                # (3.2.1): Obtain the number of arguments (mathspeak: parameters) required for each function:
                number_of_arguments_per_function = function.__code__.co_argcount

                # (3.2.2): Using the number of parameters, choose them randomly from the interval [-5, 5] to parametrize the function:
                function_parameters = np.round(np.random.uniform(-5, 5, size = number_of_arguments_per_function - 2))

                # (3.2.3): Obtain the result by passing in the required arguments:
                new_expression = function(chosen_expression, 1, *function_parameters)
            
            available_expressions.append(new_expression)

        # (X): In B(•, •) operations, we need two inputs:
        else:

            # (X): We find the two inputs to the B(•, •) operator:
            first_chosen_expression, second_chosen_expression = random.sample(available_expressions, 2)
            
            # (X): Perform a set union...
            used_independent_variables.update(set([
                first_chosen_expression,
                second_chosen_expression
                ]) & set(sympy_symbols))

            # (X): Chose a random arithmetic operation between the two:
            chosen_binary_operation = np.random.choice([sp.Add, sp.Mul, sp.Pow])
            
            # (X): Actually perform the binary operation:            
            new_expression = chosen_binary_operation(first_chosen_expression, second_chosen_expression)
            
            # (X): Add the resultant expression to the list:
            available_expressions.append(new_expression)

    # (X): The last element of `available_expressions` is the most recently-updated expression:
    final_expression = available_expressions[-1]

    # (X): "set quotient" (?) to determine if the loop missed any independent variables:
    unused_variable = set(sympy_symbols) - used_independent_variables   

    # (X): If there *are* unused variables:
    if unused_variable is not None:

        # (X): Initialize iteration over unused variables...
        for unused_variable in unused_variable:

            # (X): ...and force their inclusion into the final expression:
            final_expression += unused_variable

    return final_expression

In [60]:
sympy_generate_random_function(sympy_symbols, 3)

x_1*(-4.0*x_2**2 - 4.0*x_2 - 5.0) + x_3

In [61]:
sympy_generate_random_function(sympy_symbols, 3)

3.0*x_1**3 + 4.0*x_1**2 + 5.0*x_1 - 3.0

In [62]:
sympy_generate_random_function(sympy_symbols, 3)

x_1*x_3 + x_2

In [63]:
sympy_generate_random_function(sympy_symbols, 3)

x_1**x_2 + x_3

In [64]:
sympy_generate_random_function(sympy_symbols, 3)

5.0*x_2**2 + 3.0*x_2 + 5.0

In [65]:
sympy_generate_random_function(sympy_symbols, 3)

x_1*x_2*x_3

Seems like it works. Remarkable!